<a href="https://colab.research.google.com/github/PavankumarUppar/DLL/blob/main/Program4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Load and preprocess the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Reshape and normalize the input images
train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255

# One-hot encode the labels
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# Define the CNN architecture
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(train_images, train_labels, epochs=5, batch_size=64, validation_data=(test_images, test_labels))

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f'Test accuracy: {test_acc}')


11490434/11490434 [==============================] - 0s 0us/step
Epoch 1/5
938/938 [==============================] - 61s 64ms/step - loss: 0.1717 - accuracy: 0.9472 - val_loss: 0.0413 - val_accuracy: 0.9861
Epoch 2/5
938/938 [==============================] - 52s 55ms/step - loss: 0.0495 - accuracy: 0.9846 - val_loss: 0.0339 - val_accuracy: 0.9892
Epoch 3/5
938/938 [==============================] - 51s 55ms/step - loss: 0.0329 - accuracy: 0.9896 - val_loss: 0.0410 - val_accuracy: 0.9867
Epoch 4/5
938/938 [==============================] - 51s 54ms/step - loss: 0.0266 - accuracy: 0.9917 - val_loss: 0.0367 - val_accuracy: 0.9880
Epoch 5/5
313/313 [==============================] - 3s 9ms/step - loss: 0.0294 - accuracy: 0.9912
Test accuracy: 0.9911999702453613
